<h1>Analisis de los datos para limpieza</h1>

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
data = pd.read_csv('data\Empresas_mas_grandes_del_pais_2018.csv', delimiter=';', decimal=',')

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data.isnull().mean()

In [ ]:
data.isnull().mean().sum()

Como se puede ver el 2% de nuestras casillas están vacias, y la mayoria de los vacios pertenecen a datos que nos podrían ayudar con el analisis, se tomaran decisiones para reducir ese porcentaje de nulos.

In [ ]:
data['NIT'].count()

In [ ]:
len(data['NIT'].unique().tolist())

In [ ]:
data['RAZON SOCIAL'].count()

In [ ]:
len(data['RAZON SOCIAL'].unique().tolist())

Existen 68 empresas repetidas, con un analisis posterior unificaremos esas filas.

In [ ]:
data.SUPERVISOR.unique()

In [ ]:
data.REGIÓN.unique()

In [ ]:
data['DEPARTAMENTO DOMICILIO'].unique()

In [ ]:
data['CIUDAD DOMICILIO'].unique()

In [ ]:
data.CIIU.unique()

In [ ]:
data.MACROSECTOR.unique()

In [ ]:
data['GRUPO EN NIIF'].unique()

<h1>Preparación de los datos</h1>

<h3>Preparación para SUPERVISORES</h3>

Cambiamos el typo de SUPERSUCIEDADES a SUPERSOCIEDADES

In [ ]:
data.loc[data['SUPERVISOR'].str.contains('SUPERSUCIEDADES'), 'SUPERVISOR'] = 'SUPERSOCIEDADES'

<h3>Preparación para REGIÓN</h3>

Eliminamos el nulo de REGIÓN poniendo la región que le corresponde a su departamento registrado y luego se soluciona el typo que hay entre Costa Atlantica a Costa Atlántica.

In [ ]:
data['REGIÓN'].fillna('Costa Pacífica', inplace=True)

In [ ]:
data.loc[data['REGIÓN'].str.contains('Costa Atlantica'), 'REGIÓN'] = 'Costa Atlántica'

<h3>Preparación para DEPARTAMENTO DOMICILIO</h3>

Llenamos los nulos con la información que hay en CIUDAD DOMICILO, cambiamos el typo que hay entre BOGOTA D.C. a BOGOTÁ D.C.

In [ ]:
for index in data["No."]:
    if pd.isnull(data['DEPARTAMENTO DOMICILIO']).iloc[index-1]:
        data.loc[index-1, 'DEPARTAMENTO DOMICILIO'] = data['CIUDAD DOMICILIO'].str.rsplit('-', n=1, expand=True)[1][index-1]

In [ ]:
data.loc[data['DEPARTAMENTO DOMICILIO'].str.contains('BOGOTÁ D.C.'), 'DEPARTAMENTO DOMICILIO'] = 'BOGOTA D.C.'

<h3>Preparación para CIUDAD DOMICILIO</h3>

Remplazamos todos los nulos con la ciudad que le corresponde al departamento y luego quitamos el departamento adicionado al nombre de la ciudad.

In [ ]:
data.loc[pd.isnull(data['CIUDAD DOMICILIO']) & data['DEPARTAMENTO DOMICILIO'].str.contains('RISARALDA'), 'CIUDAD DOMICILIO'] = 'PEREIRA'

In [ ]:
data.loc[pd.isnull(data['CIUDAD DOMICILIO']) & data['DEPARTAMENTO DOMICILIO'].str.contains('BOGOTA D.C.'), 'CIUDAD DOMICILIO'] = 'BOGOTA D.C.'

In [ ]:
data.loc[pd.isnull(data['CIUDAD DOMICILIO']) & data['DEPARTAMENTO DOMICILIO'].str.contains('ANTIOQUIA'), 'CIUDAD DOMICILIO'] = 'MEDELLIN'

In [ ]:
data.loc[pd.isnull(data['CIUDAD DOMICILIO']) & data['DEPARTAMENTO DOMICILIO'].str.contains('VALLE'), 'CIUDAD DOMICILIO'] = 'CALI'

In [ ]:
for index in data["No."]:
    data.loc[index-1, 'CIUDAD DOMICILIO'] = data['CIUDAD DOMICILIO'].str.rsplit('-', n=1, expand=True)[0][index-1]
    data.loc[index-1, 'CIUDAD DOMICILIO'] = data['CIUDAD DOMICILIO'].str.replace('-', ' ')[index-1]

<h3>Preparación para CIIU</h3>

Eliminamos los nulos debido a que el análisis del modelo será sobre esos sectores y crearía sesgo ponerle alguna otra cosa.

In [ ]:
data = data[pd.notnull(data['CIIU'])]

<h3>Preparación para MACROSECTOR</h3>

Se asignan los valores de macroservicio a los nulos tomando en cuenta el CIIU.

In [ ]:
filtro = data[pd.isnull(data["MACROSECTOR"])]
filtro.head()

In [ ]:
# If the ciiu contains g4645 then the macrosector is "COMERCIO"
data.loc[data['CIIU'].str.contains('G4645'), 'MACROSECTOR'] = 'COMERCIO'

# If the ciiu contains g4641 then the macrosector is "COMERCIO"
data.loc[data['CIIU'].str.contains('G4641'), 'MACROSECTOR'] = 'COMERCIO'

# If the ciiu contains a0125 then the macrosector is "AGROPECUARIO"
data.loc[data['CIIU'].str.contains('A0125'), 'MACROSECTOR'] = 'AGROPECUARIO'

In [ ]:
data['MACROSECTOR'].unique()

In [ ]:
data.isnull().mean()

<h3>Preparación para ACTIVOS/PASIVOS/PATRIMONIO</h3>

Primero convertimos la columna TOTAL PASIVOS 2017 a float quitando el caracter $

In [ ]:
data.loc[data['TOTAL PASIVOS 2017'].str.contains('$', regex=False), 'TOTAL PASIVOS 2017'] = data['TOTAL PASIVOS 2017'].str.replace('$', '', regex=False).str.replace(' ', '', regex=False)
data['TOTAL PASIVOS 2017'] = pd.to_numeric(data['TOTAL PASIVOS 2017'], errors='coerce')

In [ ]:
data.infer_objects().dtypes
data.dtypes

Se aplica la formula del patrimonio para hallar los valores faltantes.<br>
<i>Patrimonio = Activos - Pasivos</i>

In [ ]:
nan_rows = data[data.isnull().T.any().T]

In [ ]:
print(nan_rows.shape)
nan_rows.head()

In [ ]:
# If PATRIMONIO is null que calculate it with PATRIMONIO = ACTIVOS - PASIVOS
data.loc[data['TOTAL PATRIMONIO 2018'].isnull(), 'TOTAL PATRIMONIO 2018'] = data['TOTAL ACTIVOS 2018'] - data['TOTAL PASIVOS 2018']

# If ACTIVOS is null que calculate it with ACTIVOS = PASIVOS + PATRIMONIO
data.loc[data['TOTAL ACTIVOS 2018'].isnull(), 'TOTAL ACTIVOS 2018'] = data['TOTAL PASIVOS 2018'] + data['TOTAL PATRIMONIO 2018']

# If PASIVOS is null que calculate it with PASIVOS = ACTIVOS - PATRIMONIO
data.loc[data['TOTAL PASIVOS 2018'].isnull(), 'TOTAL PASIVOS 2018'] = data['TOTAL ACTIVOS 2018'] - data['TOTAL PATRIMONIO 2018']


In [ ]:
# If PATRIMONIO is null que calculate it with PATRIMONIO = ACTIVOS - PASIVOS
data.loc[data['TOTAL PATRIMONIO 2017'].isnull(), 'TOTAL PATRIMONIO 2017'] = data['TOTAL ACTIVOS 2017'] - data['TOTAL PASIVOS 2017']

# If ACTIVOS is null que calculate it with ACTIVOS = PASIVOS + PATRIMONIO
data.loc[data['TOTAL ACTIVOS 2017'].isnull(), 'TOTAL ACTIVOS 2017'] = data['TOTAL PASIVOS 2017'] + data['TOTAL PATRIMONIO 2017']

# If PASIVOS is null que calculate it with PASIVOS = ACTIVOS - PATRIMONIO
data.loc[data['TOTAL PASIVOS 2017'].isnull(), 'TOTAL PASIVOS 2017'] = data['TOTAL ACTIVOS 2017'] - data['TOTAL PATRIMONIO 2017']

In [ ]:
nan_rows = data[data.isnull().T.any().T]

In [ ]:
print(nan_rows.shape)
nan_rows.head()

In [ ]:
data.isnull().mean()

<h3>Preparación para ingresos operacionales y ganancia</h3>

Debido a que estos datos hacen una gran contribución al modelo y no hay forma de predecirlos, vamos a eliminar las filas que no tienen este valor.

In [ ]:
data = data[pd.notnull(data['INGRESOS OPERACIONALES\n2018*'])]

In [ ]:
data = data[pd.notnull(data['INGRESOS OPERACIONALES\n2017*'])]

In [ ]:
data = data[pd.notnull(data['GANANCIA (PERDIDA) 2018'])]

In [ ]:
data = data[pd.notnull(data['GANANCIA (PERDIDA) 2017'])]

<h3>Preparación empresas repetidas</h3>

Dropeamos las empresas repetidas

In [ ]:
data = data.drop_duplicates(subset=['NIT'])

In [ ]:
data.isnull().mean()